In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(9, 0), (13, 1), (11, 0), (9, 3), (10, 1), (12, 3), (14, 3), (9, 1)]
[(11, 1), (12, 1), (14, 1), (14, 0), (13, 3), (11, 2), (12, 2), (14, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 1000
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[1000, 0],
   [0, 0],
   [8.0, 7.502],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.502],
   [2.601, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 1000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.502 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.502 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.601 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer,NaN,1000.0,514.0,1514.0,2
RandomPlayer2,486.0,1000.0,NaN,1486.0,1
YourName,0.0,NaN,0.0,0.0,0


In [12]:
from Test_Players import *
from B import *

# Saint

In [13]:
class Saint_Sceptic_25(Player):
    def __init__(self, name):
        super().__init__(name)
        self.prob_threshold = 0.25
        self.player_cards = []
    
    def putCard(self, declared_card):
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        available_cards = []
        for card in self.cards:
            if declared_card is None or card[0] >= declared_card[0]:
                available_cards.append(card)
        available_cards = sorted(available_cards, key=lambda x: x[0])
        if len(available_cards) > 0:
            true_card = available_cards[0]
            called_card = available_cards[0]
        else:
            available_cards = self.cards
            available_cards = sorted(available_cards, key=lambda x: x[0])
            true_card = available_cards[0]
            if declared_card[0] == 14:
                called_card = (14, np.random.randint(0, 4))
            else:
                called_card = (np.random.randint(declared_card[0], 15), np.random.randint(0, 4))
        self.player_cards.append(true_card)
        return true_card, called_card
    
    def get_probability(self, opponent_declaration):
        pass
    
    def checkCard(self, opponent_declaration):
        card_probability = self.get_probability(opponent_declaration)
        if card_probability < self.prob_threshold:
            return True
        return False
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log = True):
        print(revealedCard)

In [14]:
class Saint_Sceptic_50(Player):
    def __init__(self, name):
        super().__init__(name)
        self.prob_call = 0.5
    
    def putCard(self, declared_card):
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        available_cards = []
        for card in self.cards:
            if declared_card is None or card[0] >= declared_card[0]:
                available_cards.append(card)
        available_cards = sorted(available_cards, key=lambda x: x[0])
        if len(available_cards) > 0:
            true_card = available_cards[0]
            called_card = available_cards[0]
        else:
            available_cards = self.cards
            available_cards = sorted(available_cards, key=lambda x: x[0])
            true_card = available_cards[0]
            if declared_card[0] == 14:
                called_card = (14, np.random.randint(0, 4))
            else:
                called_card = (np.random.randint(declared_card[0], 15), np.random.randint(0, 4))
        return true_card, called_card
    
    def checkCard(self, opponent_declaration):
        random_float = np.random.rand()
        if random_float <= self.prob_call:
            return True
        return False
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log = True):
        if iChecked and iDrewCards:
            self.prob_call /= 2
        elif iChecked and not iDrewCards:
            self.prob_call *= 2
            if self.prob_call > 1.0:
                self.prob_call = 1.0

In [15]:
class Saint_Sceptic_75(Player):
    def __init__(self, name):
        super().__init__(name)
        self.prob_call = 0.5
    
    def putCard(self, declared_card):
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        available_cards = []
        for card in self.cards:
            if declared_card is None or card[0] >= declared_card[0]:
                available_cards.append(card)
        available_cards = sorted(available_cards, key=lambda x: x[0])
        if len(available_cards) > 0:
            true_card = available_cards[0]
            called_card = available_cards[0]
        else:
            available_cards = self.cards
            available_cards = sorted(available_cards, key=lambda x: x[0])
            true_card = available_cards[0]
            if declared_card[0] == 14:
                called_card = (14, np.random.randint(0, 4))
            else:
                called_card = (np.random.randint(declared_card[0], 15), np.random.randint(0, 4))
        return true_card, called_card
    
    def checkCard(self, opponent_declaration):
        random_float = np.random.rand()
        if random_float <= self.prob_call:
            return True
        return False
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log = True):
        if iChecked and iDrewCards:
            self.prob_call /= 2
        elif iChecked and not iDrewCards:
            self.prob_call *= 2
            if self.prob_call > 1.0:
                self.prob_call = 1.0

In [16]:
class Saint_Sceptic_Adaptable(Player):
    def __init__(self, name):
        super().__init__(name)
        self.prob_threshold = 0.5
    
    def putCard(self, declared_card):
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        available_cards = []
        for card in self.cards:
            if declared_card is None or card[0] >= declared_card[0]:
                available_cards.append(card)
        available_cards = sorted(available_cards, key=lambda x: x[0])
        if len(available_cards) > 0:
            true_card = available_cards[0]
            called_card = available_cards[0]
        else:
            available_cards = self.cards
            available_cards = sorted(available_cards, key=lambda x: x[0])
            true_card = available_cards[0]
            if declared_card[0] == 14:
                called_card = (14, np.random.randint(0, 4))
            else:
                called_card = (np.random.randint(declared_card[0], 15), np.random.randint(0, 4))
        return true_card, called_card
    
    def checkCard(self, opponent_declaration):
        random_float = np.random.rand()
        if random_float <= self.prob_call:
            return True
        return False
    
    def getCheckFeedback(self, checked, iChecked, iDrewCards, revealedCard, noTakenCards, log = True):
        if iChecked and iDrewCards:
            self.prob_call /= 2
        elif iChecked and not iDrewCards:
            self.prob_call *= 2
            if self.prob_call > 1.0:
                self.prob_call = 1.0

# Dabbler

# The Coin Flipper

# Pathological

# Pusher

In [17]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ["Saint_Believer", "SAINT", Saint_Believer],
           ["Saint_Once_in_a_While", "SAINT", Saint_Once_in_a_While],
           ["Saint_Two_Face", "SAINT", Saint_Two_Face],
           ["Saint_Accountant", "SAINT", Saint_Accountant],
           ["Saint_Nervous", "SAINT", Saint_Nervous],
           ["Saint_Collector", "SAINT", Saint_Collector],
           ["Dabbler_Believer", "DABBLER", Dabbler_Believer],
           ["Dabbler_Once_in_a_While", "DABBLER", Dabbler_Once_in_a_While],
           ["Dabbler_Two_Face", "DABBLER", Dabbler_Two_Face],
           ["Dabbler_Accountant", "DABBLER", Dabbler_Accountant],
           ["Dabbler_Nervous", "DABBLER", Dabbler_Nervous],
           ["Dabbler_Collector", "DABBLER", Dabbler_Collector],
           ["Coin_Flipper_Believer", "COIN_FLIPPER", Coin_Flipper_Believer],
           ["Coin_Flipper_Once_in_a_While", "COIN_FLIPPER", Coin_Flipper_Once_in_a_While],
           ["Coin_Flipper_Two_Face", "COIN_FLIPPER", Coin_Flipper_Two_Face],
           ["Coin_Flipper_Accountant", "COIN_FLIPPER", Coin_Flipper_Accountant],
           ["Coin_Flipper_Nervous", "COIN_FLIPPER", Coin_Flipper_Nervous],
           ["Coin_Flipper_Collector", "COIN_FLIPPER", Coin_Flipper_Collector],
           ["Pathological_Believer", "PATHOLOGICAL", Pathological_Believer],
           ["Pathological_Once_in_a_While", "PATHOLOGICAL", Pathological_Once_in_a_While],
           ["Pathological_Two_Face", "PATHOLOGICAL", Pathological_Two_Face],
           ["Pathological_Accountant", "PATHOLOGICAL", Pathological_Accountant],
           ["Pathological_Nervous", "PUSHER", Pathological_Nervous],
           ["Pathological_Collector", "PUSHER", Pathological_Collector],
           ["Pusher_Believer", "PUSHER", Pusher_Believer],
           ["Pusher_Once_in_a_While", "PUSHER", Pusher_Once_in_a_While],
           ["Pusher_Two_Face", "PUSHER", Pusher_Two_Face],
           ["Pusher_Accountant", "PUSHER", Pusher_Accountant],
           ["Pusher_Nervous", "PUSHER", Pusher_Nervous],
           ["Pusher_Collector", "PUSHER", Pusher_Collector],
           ["B_1", "B_PLAYER", Blayer],
           ["B_2", "B_PLAYER", Blayer2],
           ]

compare_all(agent_strategies)

,RandomPlayer,YourName,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,Random_Never_Accuse,Random_Always_Accuse,Mimic,Saint_Believer,Saint_Once_in_a_While,...,Pusher_Believer,Pusher_Once_in_a_While,Pusher_Two_Face,Pusher_Accountant,Pusher_Nervous,Pusher_Collector,B_1,B_2,|| Total Wins ||,|| Dominated Players ||
Saint_Nervous,1000.0,1000.0,1000.0,1000.0,1000.0,985.0,1000.0,968.0,589.0,760.0,...,926.0,963.0,1000.0,992.0,906.0,926.0,653.0,643.0,35190.0,39
Dabbler_Nervous,660.0,1000.0,752.0,767.0,725.0,956.0,603.0,990.0,867.0,704.0,...,827.0,751.0,682.0,677.0,759.0,791.0,688.0,672.0,30250.0,38
Saint_Accountant,951.0,1000.0,999.0,1000.0,994.0,971.0,1000.0,983.0,239.0,317.0,...,912.0,917.0,923.0,924.0,906.0,895.0,197.0,178.0,29817.0,32
Dabbler_Two_Face,885.0,1000.0,985.0,996.0,979.0,981.0,989.0,984.0,381.0,204.0,...,918.0,925.0,839.0,881.0,923.0,832.0,91.0,101.0,29255.0,30
Saint_Two_Face,998.0,1000.0,999.0,1000.0,1000.0,973.0,1000.0,999.0,123.0,71.0,...,874.0,918.0,991.0,970.0,889.0,873.0,120.0,115.0,29126.0,31
Dabbler_Accountant,610.0,1000.0,985.0,997.0,972.0,979.0,988.0,973.0,567.0,455.0,...,932.0,907.0,849.0,884.0,906.0,909.0,413.0,444.0,29084.0,31
Pusher_Accountant,381.0,1000.0,929.0,988.0,887.0,994.0,230.0,986.0,928.0,697.0,...,966.0,954.0,817.0,NaN,892.0,954.0,760.0,787.0,27891.0,29
Coin_Flipper_Accountant,351.0,1000.0,957.0,986.0,934.0,980.0,918.0,988.0,585.0,391.0,...,925.0,902.0,776.0,808.0,912.0,910.0,467.0,518.0,27002.0,29
RandomPlayer,NaN,1000.0,930.0,976.0,909.0,984.0,576.0,989.0,416.0,174.0,...,932.0,894.0,601.0,619.0,912.0,835.0,143.0,168.0,25997.0,27
Coin_Flipper_Two_Face,501.0,1000.0,933.0,981.0,885.0,975.0,583.0,993.0,438.0,181.0,...,916.0,903.0,621.0,624.0,902.0,798.0,159.0,149.0,25982.0,28
